In [ ]:
!pip install pystan==2.19.1.1
!pip install prophet


     |████████████████████████████████| 71kB 3.5MB/s 
     |████████████████████████████████| 51kB 5.8MB/s 
     |████████████████████████████████| 184kB 8.9MB/s 
  Created wheel for prophet: filename=prophet-1.0.1-cp37-none-any.whl size=6639590 sha256=91e586733965afdafb5e6f6ec3bcb03deaac4ca639088a663ef31b513dea7232
  Stored in directory: /root/.cache/pip/wheels/92/1d/02/bc0a91bd3a84e060722075d2b0b68e875e37ec7dc49451d513
Successfully built prophet
ERROR: fbprophet 0.7.1 has requirement cmdstanpy==0.9.5, but you'll have cmdstanpy 0.9.68 which is incompatible.
  Found existing installation: cmdstanpy 0.9.5
    Uninstalling cmdstanpy-0.9.5:
      Successfully uninstalled cmdstanpy-0.9.5


In [ ]:
import pandas as pd
import numpy as np 
from prophet import Prophet

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/xxiang27/sikkaAIHackathon/main/data_file.csv')
df.head()

,id,month,year,visits,no_of_appts,production
0,34,3,2014,1,960,76.55
1,34,4,2014,488,1089,106333.80
2,34,5,2014,831,993,192230.40
3,34,6,2014,753,884,163515.12
4,34,7,2014,721,797,160185.90


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27170 entries, 0 to 27169
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           27170 non-null  int64  
 1   month        27170 non-null  int64  
 2   year         27170 non-null  int64  
 3   visits       27170 non-null  int64  
 4   no_of_appts  27170 non-null  int64  
 5   production   27170 non-null  float64
dtypes: float64(1), int64(5)
memory usage: 1.2 MB


In [ ]:
df.describe()

,id,month,year,visits,no_of_appts,production
count,27170.000000,27170.000000,27170.000000,27170.000000,27170.000000,2.717000e+04
mean,141.936069,6.582149,2015.817372,567.629923,272.300184,1.810539e+05
std,81.682146,3.457194,3.018620,380.785944,380.348904,1.340010e+05
min,1.000000,1.000000,2010.000000,1.000000,1.000000,-1.592922e+05
25%,73.000000,4.000000,2013.000000,321.000000,18.000000,9.867377e+04
50%,143.000000,7.000000,2016.000000,479.000000,144.000000,1.498361e+05
75%,210.000000,10.000000,2018.000000,724.000000,403.000000,2.239591e+05
max,284.000000,12.000000,2020.000000,3229.000000,4147.000000,1.230910e+06


In [ ]:
def turntoDate(df):
  lst = []
  for i in range(len(df)):
    lst.append(pd.Timestamp(year = df.loc[i, 'year'], month = df.loc[i, 'month'], day=None))
  df['date'] = lst
  return df

In [ ]:
df['day'] = 1
dates = pd.to_datetime(df[['month', 'year','day']].copy())
dates.head()

0   2014-03-01
1   2014-04-01
2   2014-05-01
3   2014-06-01
4   2014-07-01
dtype: datetime64[ns]

In [ ]:
df_final = pd.concat([dates, df],axis=1)
df_final.columns.values[0] = 'date'
df_final.head()

,date,id,month,year,visits,no_of_appts,production,day
0,2014-03-01,34,3,2014,1,960,76.55,1
1,2014-04-01,34,4,2014,488,1089,106333.80,1
2,2014-05-01,34,5,2014,831,993,192230.40,1
3,2014-06-01,34,6,2014,753,884,163515.12,1
4,2014-07-01,34,7,2014,721,797,160185.90,1


In [ ]:
# separate dataframe into separate dataframes for each practice 

import math 

def separate(df):
  '''takes a dataframe returns a list of dataframes, 
  one for each unique value under the id column'''
  lst = []
  grouped = df.groupby(df.id)
  for i in range(1, 285):
    lst.append(grouped.get_group(i))
  return lst 

def train_test_split(lst, add_test):
  '''takes a list of dataframes and returns the time series train and test split based on the percent'''
  test_set = []
  for df in lst:
    df_add = df.tail(add_test).copy()
    test_set.append(df_add)
    df.drop(df.tail(add_test).index, inplace = True)
  return lst, test_set

def find_final(lst):
  months_lst = []
  years_lst = []
  weird_ids = []
  for df in lst: 
    month = df.tail(1).iloc[0,2]
    months_lst.append(month)
    year = df.tail(1).iloc[0,3]
    years_lst.append(year)
    if month != 12 or year != 2020:
      weird_ids.append(df.tail(1).iloc[0,1])
  return months_lst, years_lst, weird_ids




In [ ]:
df_final.tail(1).iloc[0,0]

Timestamp('2020-12-01 00:00:00')

In [ ]:
train_lst = separate(df_final)
months, years, weird = find_final(train_lst)

In [ ]:
df_final.head()

,date,id,month,year,visits,no_of_appts,production,day
0,2014-03-01,34,3,2014,1,960,76.55,1
1,2014-04-01,34,4,2014,488,1089,106333.80,1
2,2014-05-01,34,5,2014,831,993,192230.40,1
3,2014-06-01,34,6,2014,753,884,163515.12,1
4,2014-07-01,34,7,2014,721,797,160185.90,1


In [ ]:
df_train, df_test = train_test_split(train_lst, 4)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

In [ ]:
# create a differenced series
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return pd.Series(diff)
 
 # transform series into train and test sets for supervised learning
def prepare_data(series, n_test, n_lag, n_seq):
	# extract raw values
	raw_values = series.values
	# transform data to be stationary
	diff_series = difference(raw_values, 1)
	diff_values = diff_series.values
	diff_values = diff_values.reshape(len(diff_values), 1)
	# rescale values to -1, 1
	scaler = MinMaxScaler(feature_range=(-1, 1))
	scaled_values = scaler.fit_transform(diff_values)
	scaled_values = scaled_values.reshape(len(scaled_values), 1)
	# transform into supervised learning problem X, y
	supervised = series_to_supervised(scaled_values, n_lag, n_seq)
	supervised_values = supervised.values
	# split into train and test sets
	# train, test = supervised_values[0:-n_test], supervised_values[-n_test:]
	train, test = supervised_values[0:], supervised_values[-n_test:]
	return scaler, train, test

# fit an LSTM network to training data
def fit_lstm(train, n_lag, n_seq, n_batch, nb_epoch, n_neurons):
	# reshape training into [samples, timesteps, features]
	X, y = train[:, 0:n_lag], train[:, n_lag:]
	X = X.reshape(X.shape[0], 1, X.shape[1])
	# design network
	model = Sequential()
	model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True))
	model.add(Dense(y.shape[1]))
	model.compile(loss='mean_squared_error', optimizer='adam')
	# fit network
	for i in range(nb_epoch):
		model.fit(X, y, epochs=1, batch_size=n_batch, verbose=0, shuffle=False)
		model.reset_states()
	return model

# make one forecast with an LSTM,
def forecast_lstm(model, X, n_batch):
	# reshape input pattern to [samples, timesteps, features]
	X = X.reshape(1, 1, len(X))
	# make forecast
	forecast = model.predict(X, batch_size=n_batch)
	# convert to array
	return [x for x in forecast[0, :]]

# evaluate the persistence model
def make_forecasts(model, n_batch, train, test, n_lag, n_seq):
	forecasts = list()
	for i in range(len(test)):
		X, y = test[i, 0:n_lag], test[i, n_lag:]
		print(X)
    # make forecast
		forecast = forecast_lstm(model, X, n_batch)
		# store the forecast
		forecasts.append(forecast)
    
	return forecasts

# invert differenced forecast
def inverse_difference(last_ob, forecast):
	# invert first forecast
	inverted = list()
	inverted.append(forecast[0] + last_ob)
	# propagate difference forecast using inverted first value
	for i in range(1, len(forecast)):
		inverted.append(forecast[i] + inverted[i-1])
	return inverted
# inverse data transform on forecasts
def inverse_transform(series, forecasts, scaler, n_test):
	inverted = list()
	for i in range(len(forecasts)):
		# create array from forecast
		forecast = np.array(forecasts[i])
		forecast = forecast.reshape(1, len(forecast))
		# invert scaling
		inv_scale = scaler.inverse_transform(forecast)
		inv_scale = inv_scale[0, :]
		# invert differencing
		index = len(series) - n_test + i - 1
		last_ob = series.values[index]
		inv_diff = inverse_difference(last_ob, inv_scale)
		# store
		inverted.append(inv_diff)
	return inverted
# convert time series into supervised learning problem
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [ ]:
def make_predictions(df_train):
  make_predictions = {}
  for df in train_lst:
    X = df[['date', 'production']]
    id = df.iloc[0, 1]
    X.rename(columns={"date": "ds", "production": "y"}, inplace = True)
    
    # configure
    n_lag = 1
    n_seq = 4
    n_test = 1
    X = X.drop(columns = ['ds'])
    series = X
    scaler, train, test = prepare_data(series, n_test, n_lag, n_seq)
    # fit model
    model = fit_lstm(train, 1, 4, 1, 1500, 1)
    # make forecasts
    forecasts = make_forecasts(model, 1, train, test, 1, 4)
    # inverse transform forecasts and test
    forecasts = inverse_transform(series, forecasts, scaler, n_test+2)
    
    make_predictions[id] = forecasts[0]
  return make_predictions 
    


In [ ]:
predictions = make_predictions(df_train)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



[-0.68680792]


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



[0.14005581]


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



...

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



[-0.69779721]


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



[-0.86412571]


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
 def MAPE(y_true, y_pred):
	return np.mean(np.abs((y_true - y_pred)/y_true))*100

# need to make both of them dictionaries 
def score(predictions, actual):
  scores = []
  cate = [0,0,0,0]
  for i in range(1, 285):
    score = MAPE(actual[i], predictions[i])
    scores.append(score)
    if score <= 5:
      cate[0] += 1
    elif score > 5 and score <=10:
      cate[1] += 1
    elif score >10 and score <= 15:
      cate[2] += 1
    else:
      cate[3] += 1
  total_score = (cate[0]/284 * 2) + (cate[1]/284 * 1) + (cate[2]/284 * -1) + (cate[3]/284 * -2)
  return scores, total_score 

def prepare_test(df_test, name):
  actual = {}
  for df in df_test:
    id = df.iloc[0,1]
    actual[id] = df[name].to_numpy()
  return actual 

In [ ]:
actual = prepare_test(df_test)

In [ ]:
scores, total = score(predictions, actual)

In [ ]:
total

In [ ]:
plt.hist(scores)

In [ ]:
def prepare_train(df_train):
  train = {}
  for df in df_train:
    id = df.iloc[0,1]
    train[id] = df
  return train

def get_data_counts(train):
  counts = []
  for i in range(1, 285):
    df = train[i]
    counts.append(len(df))
  return counts


In [ ]:
counts = get_data_counts(prepare_train(df_train))


In [ ]:
plt.scatter(counts, scores)

In [ ]:
plt.hist(counts)

In [ ]:
np.mean(scores)